# DrakeVisualizer.jl
This notebook demonstrates interacting with the Drake Visualizer app from Julia. To run it, you'll need to:

* install Drake: http://drake.mit.edu/
* add the Drake lcmtypes to your python path: 
    * `export PYTHONPATH="$PYTHONPATH:/path/to/drake-distro/build/lib/python2.7/dist-packages"`
* install the DrakeVisualizer.jl package:
    * `Pkg.clone("https://github.com/rdeits/DrakeVisualizer.jl.git")`
* launch the `drake-visualizer` app:
    * `cd /path/to/drake-distro/build/bin`
    * `./drake-visualizer`
        

In [19]:
# Activate the DrakeVisualizer package, and import some other 
# useful functions
using DrakeVisualizer
import GeometryTypes: HyperRectangle, Vec, HomogenousMesh, SignedDistanceField
using CoordinateTransformations
import Meshing
import ColorTypes: RGBA

INFO: Precompiling module DrakeVisualizer...


In [20]:
import Interact: @manipulate

In [21]:
# First, we'll create a simple geometric object
box = HyperRectangle(Vec(0.,0,0), Vec(1.,1,1))

GeometryTypes.HyperRectangle{3,Float64}(Vec(0.0,0.0,0.0),Vec(1.0,1.0,1.0))

In [22]:
# Visualizer() causes the viewer to spawn a geometry or a set of geometries. 
# It returns a Visualizer, which includes all the information
# about that loaded geometry. 
# Note that the model is initially loaded in the zero configuration 
# (that is, its position and rotation are all zeros)
model = Visualizer(box);

In [23]:
# We can use draw() to tell the viewer to draw the box at a specific
# position. Translation() creates a CoordinateTransformations.Transformation
# corresponding to the given x; y; z translation.
draw(model, [Translation(1.,0,0)])

In [24]:
# We can also rotate the model by sending a different transformation
draw(model, [LinearMap(AngleAxis(pi/4, 0, 0, 1))])

In [25]:
# Let's look more at what's being done under the hood. The model
# owns a reference to a Robot object. A Robot contains a list of 
# Link objects. Each Link contains a list of GeometryData objects.
# And each GeometryData contains a single geometric primitive as
# well as information about its color and its position with
# respect to the link that contains it.
#
# Calling Visualizer() on a single geometry (the box above) results in 
# a new Robot being automatically created. That Robot has one link,
# and that Link has one GeometryData whose geometry is the box we
# provided. 
model.robot

DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HyperRectangle{3,Float64}}(GeometryTypes.HyperRectangle{3,Float64}(Vec(0.0,0.0,0.0),Vec(1.0,1.0,1.0)),CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")])

In [26]:
model.robot.links

1-element Array{DrakeVisualizer.Link,1}:
 DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HyperRectangle{3,Float64}}(GeometryTypes.HyperRectangle{3,Float64}(Vec(0.0,0.0,0.0),Vec(1.0,1.0,1.0)),CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")

In [27]:
# Here's the box we created. It's now the first (and only) geometry
# of the first (and only) link of the robot:
model.robot.links[1].geometry_data[1].geometry

GeometryTypes.HyperRectangle{3,Float64}(Vec(0.0,0.0,0.0),Vec(1.0,1.0,1.0))

In [28]:
@assert model.robot.links[1].geometry_data[1].geometry === box

In [29]:
# Now let's make some more complicated robots. We'll create a 
# new GeometryData from the box, but color it green this time.
green_box_data = GeometryData(box)
green_box_data.color = RGBA(0., 1, 0, 0.5)
model = Visualizer(green_box_data);

In [30]:
# Now let's create a blue box
blue_box_data = GeometryData(box, Translation(0,0,1.), RGBA(0,0,1,0.5))

# We can create a new Link that contains two geometries: the 
# green box and the blue box. 
link1 = Link([green_box_data; blue_box_data], "link1")
model = Visualizer(link1);

In [31]:
# And we can draw that new link wherever we like.
draw(model, [Translation(0.75, 0, 0)])

# Note that the green and blue boxes move together. That's
# because they are part of the same rigid Link. 

In [32]:
# If we want different parts of our robot to move separately,
# they need to be on separate links. Let's create a second
# Link and then a robot containing both of those links. 
red_box_data = GeometryData(box)
link2 = Link([red_box_data], "link2")
robot = Robot([link1; link2])

# When we load this new robot, the two links are both drawn 
# at position [0; 0; 0]; right on top of each other.
model = Visualizer(robot);


In [33]:
# Now that we have two links, we need to specify two transforms
# for the draw() command. The order of the transforms matches
# the order of the links in "Robot([link1; link2])"
draw(model, [Translation(0.5, 0, 0), Translation(-0.5, 0, 0)])

In [34]:
# We can now move the two links independently
@manipulate for x1 in linspace(0, 2), x2 in linspace(0, 2)
    draw(model, [Translation(x1, 0, 0); Translation(x2, 0, 0)])
end

Interact.Slider{Float64}(Signal{Float64}(0.9795918367346939, nactions=0),"x1",0.9795918367346939,linspace(0.0,2.0,50),".3f",true)

Interact.Slider{Float64}(Signal{Float64}(0.9795918367346939, nactions=0),"x2",0.9795918367346939,linspace(0.0,2.0,50),".3f",true)

nothing

In [35]:
# Of course, we can draw much more interesting geometries than 
# just simple boxes. Let's create a triangulated mesh by finding
# the 0-level set of some function. 
# 
# First, we'll define our function:
f = x -> norm(x)^2 - abs(sin(atan2(x[2], x[1])))

# Then we pick a region of interest in which to sample the function.
# This region starts at (-1, -1, -1) and extends to (1, 1, 1):
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

# Those two pieces of information are all we need to construct a robot
# geometry. For this, we'll need the GeometryData type:
geometry = GeometryData(f, lower_bound, upper_bound)
# Under the hood, this will sample f at regularly spaced points inside
# the bounding rectangle, then compute a surface that connects all the 
# points for which f(x) = 0.

# And now we can load that geometry into the visualizer
model = Visualizer(geometry)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 4952xFixedSizeArrays.Point{3,Float64},     faces: 9896xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")]),1,PyLCM.LCM(PyObject <LCM object at 0x7efbcbbad2d0>))

In [36]:
# We can even manipulate the geometry by changing the iso level. 
# By default, GeometryData constructs a mesh connecting the points 
# in space for which f(x) = 0, where 0 is called the isosurface level
# or iso level. But we can change that iso level to any number we want:

f = x -> norm(x)^2 - abs(sin(atan2(x[2], x[1])))
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

@manipulate for iso_level in linspace(-1, 1)
    geometry = GeometryData(f, lower_bound, upper_bound, iso_level)
    model = Visualizer(geometry);
end

# Note that for high iso_level values our geometry gets cut off at the
# edges. We could fix that by replacing the bounds with a bigger box. 

Interact.Slider{Float64}(Signal{Float64}(-0.02040816326530612, nactions=0),"iso_level",-0.02040816326530612,linspace(-1.0,1.0,50),".3f",true)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 4900xFixedSizeArrays.Point{3,Float64},     faces: 9792xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"link")]),1,PyLCM.LCM(PyObject <LCM object at 0x7efbcbbad300>))